In [53]:
import os
import json
import numpy as np
import torch 
import transformers
import warnings


from sklearn.model_selection import train_test_split
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

warnings.filterwarnings('ignore')

In [54]:
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [56]:
path = os.path.abspath('interim/3/train.json')

In [57]:
path = os.path.abspath('interim/3/train.json')

with open(path) as js:
    full_review = json.load(js)

In [58]:
review = [None] * len(full_review)

for i in range(len(full_review)):
    review[i] = full_review[i]['text']

## preprocessing

In [59]:
target = [None] * len(review)
main_review = [None] * len(review)

In [60]:
for i in range(len(review)):
    #delete general_impression from review
    main_review[i] = review[i]
    general_impression_idx = main_review[i].find('Общее впечатление')
    if general_impression_idx != -1:
        main_review[i] = main_review[i][:general_impression_idx]
        
    #delete everything before review
    review_idx = main_review[i].find('Отзыв: ')
    if review_idx != -1:
        main_review[i] = main_review[i][review_idx + len('Отзыв: '):]
        
    #target 
    target_idx = main_review[i].find('\n')
    target[i] = main_review[i][:target_idx]

    #delete "TEXT" and "\n" if they exist 
    TEXT_idx = main_review[i].rfind('\n')
    if TEXT_idx != -1:
        main_review[i] = main_review[i][TEXT_idx + len('\n'):]
    

In [8]:
train_review, test_review, train_target, test_target = train_test_split(main_review, target)

In [9]:
model_checkpoint = 'cointegrated/rut5-base-absum'

In [10]:
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [11]:
data = {'train':Dataset.from_dict({'summary':train_target,'text':train_review}),
     'test':Dataset.from_dict({'summary':test_target,'text':test_review})
     }

data = DatasetDict(data)

In [12]:
data

DatasetDict({
    train: Dataset({
        features: ['summary', 'text'],
        num_rows: 2200
    })
    test: Dataset({
        features: ['summary', 'text'],
        num_rows: 734
    })
})

In [33]:
max_intput_length = 1024
max_target_length = 128
min_target_length = 50

In [14]:
def preprocess_function(examples):
    type(examples)
    model_inputs = tokenizer(examples["text"], max_length=max_intput_length, truncation=True, padding=True)

    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
tokenized_data = data.map(preprocess_function, batched=True)

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

Map:   0%|          | 0/734 [00:00<?, ? examples/s]

In [16]:
data_collator = transformers.DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [17]:
training_args = transformers.Seq2SeqTrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=2,
    )

In [18]:
trainer = transformers.Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.625400,2.080383
2,2.320300,1.996558


TrainOutput(global_step=4400, training_loss=2.5970169760964135, metrics={'train_runtime': 1148.8978, 'train_samples_per_second': 3.83, 'train_steps_per_second': 3.83, 'total_flos': 3075808886784000.0, 'train_loss': 2.5970169760964135, 'epoch': 2.0})

In [46]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [45]:
text_example = data["test"]['text'][0]
print(text_example)

У моей сестры появилось такое заболевание, как экземный дерматит. Вещь не заразная, но очень неприятная как косметически, так и по ощущениям заболевшего. Врач после осмотра и анализов, назначил лечение в виде таблеток, мазей (про которые я напишу позже) и курс из двадцати уколов супрастина, про которого и пойдет речь в сегодняшнем отзыве. Продаются по рецепту, в упаковка по пять ампул в каждой. Стоимость упаковки составляет сто сорок три рубля с копейками. Пять маленьких стеклянных ампул с 1 мл лекарственного раствора в каждом. На горлышке ампул есть место слома. Они лежат в коробочке так, что если ее уронить, то ампулы не разобьются, проверено на собственном опыте. Данный раствор вводится как внутримышечно так и внутревенно. Показаниями являются аллергии и различные дерматиты. Противопоказания тоже есть, самый главный детский возраст и беременность. Дозу определяет врач в зависимости от типа и степени заболевания. Нам назначали по одной ампуле два раза в день, внутримышечно. Побочных 

In [47]:
input_ids = tokenizer.encode(
    text_example,
    return_tensors="pt",
    max_length=1024,
    truncation=True,
    ).to(device)

In [34]:
summary_text_ids = model.generate(
    input_ids=input_ids,
    bos_token_id=model.config.bos_token_id,
    eos_token_id=model.config.eos_token_id,
    max_length=max_target_length,
    min_length=min_target_length,
    num_beams=4,
)

In [35]:
summary_text_ids

tensor([[    0,   259, 19005,  1968, 18522,  1292,  6109,   313, 17327, 17743,
         16768,   311,   259,   264,   259, 14061,  1909,  5548,   411,   259,
           735,   259, 11523, 19156,  1122,  1188, 27347, 14461,   261,  1349,
           259,  2021,  1652, 11547,  1454,  7540,   637,   309,   309,   309,
           309,   309,   309,   309,   309,   309,   309,   309,   309,   309,
             1]], device='cuda:0')

In [42]:
decoded_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
decoded_text

'Противовирусный препарат "Супрастин" - Хорошее средство для лечения экземного дерматита, но только после назначения врачом!!!!!!!!!!!!!'

In [43]:
data['test']['summary'][0]

'Раствор для внутривенного и внутримышечного введения Egis "Супрастин" - Помог при экземном дерматите'